In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.widgets import Slider
import ipywidgets as w
from IPython.display import display, clear_output, HTML
import json
from pathlib import Path
from tqdm import tqdm
import cv2
from datetime import datetime

plt.ioff()  # Prevent extra figure windows

In [2]:
# Configuration
CYCLES_DIR = Path("/mnt/tcia_data/processed/4D-Lung-Cycles")
MANIFEST_PATH = CYCLES_DIR / "breathing_cycles_manifest.csv"
EXPORTS_DIR = CYCLES_DIR / "exports"
EXPORTS_DIR.mkdir(exist_ok=True)

print(f"Data directory: {CYCLES_DIR}")
print(f"Exports directory: {EXPORTS_DIR}")

Data directory: /mnt/tcia_data/processed/4D-Lung-Cycles
Exports directory: /mnt/tcia_data/processed/4D-Lung-Cycles/exports


In [3]:
if MANIFEST_PATH.exists():
    cycles_df = pd.read_csv(MANIFEST_PATH)
    print(f"Loaded {len(cycles_df)} breathing cycles")
    print(f"Patients: {cycles_df['patient_id'].nunique()}")
    print(f"Average cycles per patient: {len(cycles_df) / cycles_df['patient_id'].nunique():.1f}")
    
    # Display sample data
    display(cycles_df.head())
else:
    print(f"Manifest not found: {MANIFEST_PATH}")
    print("Please run the breathing cycle regrouping script first.")


Loaded 587 breathing cycles
Patients: 20
Average cycles per patient: 29.4


,cycle_id,patient_id,series_id,cycle_dir,num_phases,phase_range,breathing_phases,file_paths
0,100_HM10395_S111,100_HM10395,S111,/mnt/tcia_data/processed/4D-Lung-Cycles/100_HM...,10,90.0,"[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0...",['/mnt/tcia_data/processed/4D-Lung-Cycles/100_...
1,100_HM10395_S125,100_HM10395,S125,/mnt/tcia_data/processed/4D-Lung-Cycles/100_HM...,10,90.0,"[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0...",['/mnt/tcia_data/processed/4D-Lung-Cycles/100_...
2,100_HM10395_S101,100_HM10395,S101,/mnt/tcia_data/processed/4D-Lung-Cycles/100_HM...,10,90.0,"[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0...",['/mnt/tcia_data/processed/4D-Lung-Cycles/100_...
3,100_HM10395_S129,100_HM10395,S129,/mnt/tcia_data/processed/4D-Lung-Cycles/100_HM...,10,90.0,"[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0...",['/mnt/tcia_data/processed/4D-Lung-Cycles/100_...
4,100_HM10395_S104,100_HM10395,S104,/mnt/tcia_data/processed/4D-Lung-Cycles/100_HM...,10,90.0,"[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0...",['/mnt/tcia_data/processed/4D-Lung-Cycles/100_...


In [4]:
# Data exploration functions
def load_breathing_cycle(cycle_dir):
    """Load a complete breathing cycle as 4D numpy array"""
    cycle_path = Path(cycle_dir)
    
    if not cycle_path.exists():
        raise FileNotFoundError(f"Cycle directory not found: {cycle_path}")
    
    # Get all phase files
    phase_files = sorted(cycle_path.glob("phase_*.npy"))
    
    if len(phase_files) == 0:
        raise FileNotFoundError(f"No phase files found in: {cycle_path}")
    
    # Load all phases
    phases = []
    for phase_file in phase_files:
        volume = np.load(phase_file)
        phases.append(volume)
    
    # Stack into 4D array (time, z, y, x)
    breathing_cycle_4d = np.stack(phases, axis=0)
    
    return breathing_cycle_4d, phase_files

def normalize_for_display(volume, percentile_range=(1, 99)):
    """Normalize CT volume for display using lung windowing"""
    
    # CT lung windowing (better than percentile for lung tissue)
    window_center = -600  # Lung tissue center
    window_width = 1500   # Lung tissue width
    
    lower = window_center - window_width / 2
    upper = window_center + window_width / 2
    
    # Apply windowing
    volume_windowed = np.clip(volume, lower, upper)
    volume_norm = (volume_windowed - lower) / (upper - lower)
    
    return (volume_norm * 255).astype(np.uint8)

def create_breathing_video_html(cycle_4d, slice_idx=None, output_path=None, fps=2):
    """Create HTML video of breathing cycle"""
    
    if slice_idx is None:
        slice_idx = cycle_4d.shape[1] // 2  # Middle slice
    
    # Extract slice across all breathing phases
    slice_sequence = cycle_4d[:, slice_idx, :, :]  # (time, y, x)
    
    # Normalize for display
    slice_sequence_norm = normalize_for_display(slice_sequence)
    
    # Create video frames
    frames = []
    for t in range(slice_sequence_norm.shape[0]):
        frame = slice_sequence_norm[t]
        frames.append(frame)
    
    # Save as MP4 using OpenCV
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        height, width = frames[0].shape
        video_writer = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height), False)
        
        for frame in frames:
            video_writer.write(frame)
        video_writer.release()
        
        print(f"Video saved: {output_path}")
        
        # Create HTML with video player
        html_path = output_path.with_suffix('.html')
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Breathing Cycle Video</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 20px; }}
                .video-container {{ text-align: center; margin: 20px 0; }}
                video {{ border: 1px solid #ccc; }}
                .info {{ background: #f5f5f5; padding: 10px; border-radius: 5px; margin: 10px 0; }}
            </style>
        </head>
        <body>
            <h1>4D-Lung Breathing Cycle</h1>
            <div class="info">
                <p><strong>Video:</strong> {output_path.name}</p>
                <p><strong>Slice:</strong> {slice_idx}</p>
                <p><strong>Dimensions:</strong> {cycle_4d.shape}</p>
                <p><strong>Breathing phases:</strong> {cycle_4d.shape[0]}</p>
                <p><strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            </div>
            <div class="video-container">
                <video width="{width}" height="{height}" controls loop autoplay>
                    <source src="{output_path.name}" type="video/mp4">
                    Your browser does not support the video tag.
                </video>
            </div>
            <p>This video shows the breathing motion across {cycle_4d.shape[0]} respiratory phases (0% to 90% of breathing cycle).</p>
        </body>
        </html>
        """
        
        with open(html_path, 'w') as f:
            f.write(html_content)
        
        print(f"HTML player saved: {html_path}")
        return html_path
    
    return frames

def analyze_breathing_motion(cycle_4d, slice_idx=None):
    """Analyze breathing motion patterns"""
    
    if slice_idx is None:
        slice_idx = cycle_4d.shape[1] // 2
    
    slice_sequence = cycle_4d[:, slice_idx, :, :]
    
    # Calculate motion metrics
    motion_analysis = {
        'slice_index': slice_idx,
        'temporal_frames': cycle_4d.shape[0],
        'spatial_dimensions': slice_sequence.shape[1:],
        'intensity_stats': {
            'mean_per_phase': [float(slice_sequence[t].mean()) for t in range(cycle_4d.shape[0])],
            'std_per_phase': [float(slice_sequence[t].std()) for t in range(cycle_4d.shape[0])],
            'min_per_phase': [float(slice_sequence[t].min()) for t in range(cycle_4d.shape[0])],
            'max_per_phase': [float(slice_sequence[t].max()) for t in range(cycle_4d.shape[0])]
        }
    }
    
    # Calculate inter-frame differences (motion magnitude)
    motion_magnitudes = []
    for t in range(1, cycle_4d.shape[0]):
        diff = np.abs(slice_sequence[t] - slice_sequence[t-1])
        motion_magnitudes.append(float(diff.mean()))
    
    motion_analysis['motion_magnitude'] = motion_magnitudes
    motion_analysis['avg_motion_magnitude'] = float(np.mean(motion_magnitudes))
    motion_analysis['max_motion_magnitude'] = float(np.max(motion_magnitudes))
    
    return motion_analysis


In [5]:
# Interactive widgets setup
if 'cycles_df' in locals():
    patients = sorted(cycles_df['patient_id'].unique())
    
    # Patient selection
    patient_dd = w.Dropdown(
        options=patients,
        description="Patient:",
        layout=w.Layout(width="200px")
    )
    
    # Cycle selection (will be updated based on patient)
    cycle_dd = w.Dropdown(
        options=[],
        description="Cycle:",
        layout=w.Layout(width="300px")
    )
    
    # Slice selection
    slice_slider = w.IntSlider(
        value=25,
        min=0,
        max=50,
        step=1,
        description="Slice:",
        layout=w.Layout(width="300px")
    )
    
    # Video controls
    fps_slider = w.IntSlider(
        value=2,
        min=1,
        max=10,
        step=1,
        description="FPS:",
        layout=w.Layout(width="200px")
    )
    
    # Action buttons
    load_btn = w.Button(description="Load Cycle", button_style="primary")
    video_btn = w.Button(description="Create Video", button_style="success")
    analyze_btn = w.Button(description="Analyze Motion", button_style="warning")
    export_btn = w.Button(description="Export All", button_style="info")
    
    # Output areas
    cycle_info_out = w.Output()
    plot_out = w.Output()
    video_out = w.Output()
    analysis_out = w.Output()
    
    # Global variables for loaded data
    current_cycle_4d = None
    current_cycle_info = None


In [ ]:
# Widget callback functions
def update_cycles(change):
    """Update cycle dropdown based on selected patient"""
    patient_id = change['new']
    patient_cycles = cycles_df[cycles_df['patient_id'] == patient_id]
    
    cycle_options = []
    for _, row in patient_cycles.iterrows():
        cycle_id = row['cycle_id']
        series_id = row['series_id']
        cycle_options.append((f"{series_id} ({cycle_id})", row['cycle_dir']))
    
    cycle_dd.options = cycle_options
    if cycle_options:
        cycle_dd.value = cycle_options[0][1]

def load_cycle(_):
    """Load selected breathing cycle"""
    global current_cycle_4d, current_cycle_info
    
    cycle_info_out.clear_output()
    plot_out.clear_output()
    
    if not cycle_dd.value:
        with cycle_info_out:
            print("Please select a cycle")
        return
    
    cycle_dir = cycle_dd.value
    
    with cycle_info_out:
        print(f"Loading cycle from: {cycle_dir}")
        
        try:
            # Load breathing cycle
            cycle_4d, phase_files = load_breathing_cycle(cycle_dir)
            current_cycle_4d = cycle_4d
            current_cycle_info = {
                'cycle_dir': cycle_dir,
                'phase_files': phase_files,
                'patient_id': patient_dd.value
            }
            
            print(f"✓ Loaded successfully")
            print(f"Shape: {cycle_4d.shape} (time, z, y, x)")
            print(f"Temporal frames: {cycle_4d.shape[0]}")
            print(f"Spatial dimensions: {cycle_4d.shape[1:]} slices")
            print(f"Data type: {cycle_4d.dtype}")
            print(f"Memory usage: {cycle_4d.nbytes / 1024**2:.1f} MB")
            
            # Update slice slider range
            slice_slider.max = cycle_4d.shape[1] - 1
            slice_slider.value = cycle_4d.shape[1] // 2
            
            # Show sample images
            with plot_out:
                fig, axes = plt.subplots(2, 5, figsize=(15, 6))
                axes = axes.flatten()
                
                mid_slice = cycle_4d.shape[1] // 2
                slice_sequence = cycle_4d[:, mid_slice, :, :]
                slice_sequence_norm = normalize_for_display(slice_sequence)
                
                for t in range(min(10, cycle_4d.shape[0])):
                    axes[t].imshow(slice_sequence_norm[t], cmap='gray')
                    axes[t].set_title(f'Phase {t} ({t*10}%)')
                    axes[t].axis('off')
                
                # Hide unused subplots
                for i in range(cycle_4d.shape[0], 10):
                    axes[i].axis('off')
                
                plt.suptitle(f'Breathing Cycle Preview - Slice {mid_slice}')
                plt.tight_layout()
                plt.show()
                
        except Exception as e:
            print(f"✗ Error loading cycle: {e}")

def create_matplotlib_animation(cycle_4d, slice_idx=None, fps=2):
    """Create matplotlib animation that works in VS Code"""
    
    if slice_idx is None:
        slice_idx = cycle_4d.shape[1] // 2
    
    # Extract and normalize slice sequence
    slice_sequence = cycle_4d[:, slice_idx, :, :]
    
    # Apply lung windowing
    window_center = -600
    window_width = 1500
    lower = window_center - window_width / 2
    upper = window_center + window_width / 2
    
    slice_windowed = np.clip(slice_sequence, lower, upper)
    slice_norm = (slice_windowed - lower) / (upper - lower)
    
    # Create matplotlib animation
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Initial frame
    im = ax.imshow(slice_norm[0], cmap='gray', vmin=0, vmax=1)
    ax.set_title(f'Breathing Cycle - Slice {slice_idx}\nPhase 0 (0%)')
    ax.axis('off')
    
    def animate(frame):
        im.set_array(slice_norm[frame])
        phase_percent = frame * 10  # 0%, 10%, 20%, etc.
        ax.set_title(f'Breathing Cycle - Slice {slice_idx}\nPhase {frame} ({phase_percent}%)')
        return [im]
    
    # Create animation
    anim = animation.FuncAnimation(
        fig, animate, frames=slice_norm.shape[0], 
        interval=1000//fps, blit=True, repeat=True
    )
    
    plt.tight_layout()
    plt.show()
    
    return anim

def create_video(_):
    """Create matplotlib animation for VS Code"""
    video_out.clear_output()
    
    if current_cycle_4d is None:
        with video_out:
            print("Please load a cycle first")
        return
    
    with video_out:
        print("Creating breathing animation...")
        
        try:
            slice_idx = slice_slider.value
            fps = fps_slider.value
            
            # Create matplotlib animation
            anim = create_matplotlib_animation(current_cycle_4d, slice_idx=slice_idx, fps=fps)
            
            print(f"✓ Animation created successfully")
            print(f"Slice: {slice_idx}, FPS: {fps}")
            
            # Also create static multi-frame view
            slice_sequence = current_cycle_4d[:, slice_idx, :, :]
            
            # Apply lung windowing
            window_center = -600
            window_width = 1500
            lower = window_center - window_width / 2
            upper = window_center + window_width / 2
            
            slice_windowed = np.clip(slice_sequence, lower, upper)
            slice_norm = (slice_windowed - lower) / (upper - lower)
            
            # Create static overview
            fig, axes = plt.subplots(2, 5, figsize=(15, 6))
            axes = axes.flatten()
            
            for i in range(min(10, slice_norm.shape[0])):
                axes[i].imshow(slice_norm[i], cmap='gray', vmin=0, vmax=1)
                axes[i].set_title(f'Phase {i} ({i*10}%)')
                axes[i].axis('off')
            
            # Hide unused subplots
            for i in range(slice_norm.shape[0], 10):
                axes[i].axis('off')
            
            plt.suptitle(f'Static Overview - Slice {slice_idx}')
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"✗ Error creating animation: {e}")

def analyze_motion(_):
    """Analyze breathing motion patterns"""
    analysis_out.clear_output()
    
    if current_cycle_4d is None:
        with analysis_out:
            print("Please load a cycle first")
        return
    
    with analysis_out:
        print("Analyzing breathing motion...")
        
        try:
            slice_idx = slice_slider.value
            motion_analysis = analyze_breathing_motion(current_cycle_4d, slice_idx)
            
            print(f"✓ Motion analysis complete")
            print(f"Slice: {motion_analysis['slice_index']}")
            print(f"Average motion magnitude: {motion_analysis['avg_motion_magnitude']:.2f}")
            print(f"Maximum motion magnitude: {motion_analysis['max_motion_magnitude']:.2f}")
            
            # Plot motion analysis
            fig, axes = plt.subplots(2, 2, figsize=(12, 8))
            
            phases = list(range(motion_analysis['temporal_frames']))
            
            # Intensity statistics
            axes[0,0].plot(phases, motion_analysis['intensity_stats']['mean_per_phase'], 'b-', label='Mean')
            axes[0,0].fill_between(phases, 
                                 np.array(motion_analysis['intensity_stats']['mean_per_phase']) - np.array(motion_analysis['intensity_stats']['std_per_phase']),
                                 np.array(motion_analysis['intensity_stats']['mean_per_phase']) + np.array(motion_analysis['intensity_stats']['std_per_phase']),
                                 alpha=0.3)
            axes[0,0].set_title('Intensity Statistics per Phase')
            axes[0,0].set_xlabel('Breathing Phase')
            axes[0,0].set_ylabel('Intensity')
            axes[0,0].legend()
            axes[0,0].grid(True, alpha=0.3)
            
            # Motion magnitude
            motion_phases = list(range(1, motion_analysis['temporal_frames']))
            axes[0,1].plot(motion_phases, motion_analysis['motion_magnitude'], 'r-o', markersize=4)
            axes[0,1].set_title('Motion Magnitude Between Phases')
            axes[0,1].set_xlabel('Phase Transition')
            axes[0,1].set_ylabel('Average Pixel Difference')
            axes[0,1].grid(True, alpha=0.3)
            
            # Intensity range per phase
            axes[1,0].fill_between(phases,
                                 motion_analysis['intensity_stats']['min_per_phase'],
                                 motion_analysis['intensity_stats']['max_per_phase'],
                                 alpha=0.5, color='green')
            axes[1,0].set_title('Intensity Range per Phase')
            axes[1,0].set_xlabel('Breathing Phase')
            axes[1,0].set_ylabel('Intensity')
            axes[1,0].grid(True, alpha=0.3)
            
            # Summary statistics
            axes[1,1].text(0.1, 0.8, f"Temporal frames: {motion_analysis['temporal_frames']}", transform=axes[1,1].transAxes)
            axes[1,1].text(0.1, 0.7, f"Spatial dimensions: {motion_analysis['spatial_dimensions']}", transform=axes[1,1].transAxes)
            axes[1,1].text(0.1, 0.6, f"Avg motion magnitude: {motion_analysis['avg_motion_magnitude']:.3f}", transform=axes[1,1].transAxes)
            axes[1,1].text(0.1, 0.5, f"Max motion magnitude: {motion_analysis['max_motion_magnitude']:.3f}", transform=axes[1,1].transAxes)
            axes[1,1].text(0.1, 0.4, f"Slice analyzed: {motion_analysis['slice_index']}", transform=axes[1,1].transAxes)
            axes[1,1].set_title('Analysis Summary')
            axes[1,1].axis('off')
            
            plt.suptitle(f'Breathing Motion Analysis - {current_cycle_info["patient_id"]}')
            plt.tight_layout()
            plt.show()
            
            # Save analysis
            patient_id = current_cycle_info['patient_id']
            cycle_name = Path(current_cycle_info['cycle_dir']).name
            analysis_path = EXPORTS_DIR / f"{patient_id}_{cycle_name}_slice{slice_idx:03d}_analysis.json"
            
            with open(analysis_path, 'w') as f:
                json.dump(motion_analysis, f, indent=2)
            
            print(f"Analysis saved: {analysis_path}")
            
        except Exception as e:
            print(f"✗ Error in motion analysis: {e}")

def export_all(_):
    """Export videos and analysis for all slices of current cycle"""
    if current_cycle_4d is None:
        print("Please load a cycle first")
        return
    
    print("Exporting all slices...")
    
    patient_id = current_cycle_info['patient_id']
    cycle_name = Path(current_cycle_info['cycle_dir']).name
    fps = fps_slider.value
    
    # Create patient-specific export directory
    patient_export_dir = EXPORTS_DIR / patient_id / cycle_name
    patient_export_dir.mkdir(parents=True, exist_ok=True)
    
    n_slices = current_cycle_4d.shape[1]
    
    for slice_idx in tqdm(range(0, n_slices, 5), desc="Exporting slices"):  # Every 5th slice
        try:
            # Create video
            output_name = f"slice_{slice_idx:03d}"
            video_path = patient_export_dir / f"{output_name}.mp4"
            
            create_breathing_video_html(
                current_cycle_4d,
                slice_idx=slice_idx,
                output_path=video_path,
                fps=fps
            )
            
            # Create analysis
            motion_analysis = analyze_breathing_motion(current_cycle_4d, slice_idx)
            analysis_path = patient_export_dir / f"{output_name}_analysis.json"
            
            with open(analysis_path, 'w') as f:
                json.dump(motion_analysis, f, indent=2)
                
        except Exception as e:
            print(f"Error processing slice {slice_idx}: {e}")
    
    print(f"✓ Export complete: {patient_export_dir}")

# Connect callbacks
if 'cycles_df' in locals():
    patient_dd.observe(update_cycles, names='value')
    load_btn.on_click(load_cycle)
    video_btn.on_click(create_video)
    analyze_btn.on_click(analyze_motion)
    export_btn.on_click(export_all)
    
    # Initialize with first patient
    if len(patients) > 0:
        update_cycles({'new': patients[0]})

# %%
# Main UI Layout
if 'cycles_df' in locals():
    
    controls = w.VBox([
        w.HTML("<h3>4D-Lung Breathing Cycle Explorer</h3>"),
        w.HBox([patient_dd, cycle_dd]),
        w.HBox([slice_slider, fps_slider]),
        w.HBox([load_btn, video_btn, analyze_btn, export_btn]),
    ])
    
    tabs = w.Tab()
    tabs.children = [cycle_info_out, plot_out, video_out, analysis_out]
    tabs.set_title(0, "Cycle Info")
    tabs.set_title(1, "Preview")
    tabs.set_title(2, "Animation")
    tabs.set_title(3, "Motion Analysis")
    
    main_ui = w.VBox([controls, tabs])
    display(main_ui)
    
    print("\n=== Instructions ===")
    print("1. Select patient and breathing cycle")
    print("2. Click 'Load Cycle' to load the data")
    print("3. Adjust slice and FPS settings")
    print("4. Click 'Create Video' to generate HTML video")
    print("5. Click 'Analyze Motion' for motion analysis")
    print("6. Click 'Export All' to export multiple slices")



=== Instructions ===
1. Select patient and breathing cycle
2. Click 'Load Cycle' to load the data
3. Adjust slice and FPS settings
4. Click 'Create Video' to generate HTML video
5. Click 'Analyze Motion' for motion analysis
6. Click 'Export All' to export multiple slices


In [7]:
# Quick check - run this in your notebook
import cv2
from pathlib import Path

video_path = "/mnt/tcia_data/processed/4D-Lung-Cycles/exports/100_HM10395_100_HM10395_S111_slice025_2fps.mp4"

# Check file size
video_file = Path(video_path)
if video_file.exists():
    print(f"File exists: {video_file}")
    print(f"File size: {video_file.stat().st_size} bytes")
    
    # Try to read with OpenCV
    cap = cv2.VideoCapture(str(video_path))
    if cap.isOpened():
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        print(f"Video properties: {width}x{height}, {frame_count} frames, {fps} FPS")
        
        # Try to read first frame
        ret, frame = cap.read()
        if ret:
            print(f"✓ Successfully read frame: {frame.shape}")
            print(f"Frame value range: {frame.min()} to {frame.max()}")
        else:
            print("✗ Could not read frame")
        cap.release()
    else:
        print("✗ Could not open video file")
else:
    print("✗ Video file does not exist")

File exists: /mnt/tcia_data/processed/4D-Lung-Cycles/exports/100_HM10395_100_HM10395_S111_slice025_2fps.mp4
File size: 729680 bytes
Video properties: 512x512, 10 frames, 2.0 FPS
✓ Successfully read frame: (512, 512, 3)
Frame value range: 0 to 255


In [8]:
def batch_process_patients(patient_list=None, max_cycles_per_patient=3):
    """Process multiple patients in batch"""
    
    if patient_list is None:
        patient_list = cycles_df['patient_id'].unique()[:3]  # First 3 patients
    
    batch_results = []
    
    for patient_id in tqdm(patient_list, desc="Processing patients"):
        patient_cycles = cycles_df[cycles_df['patient_id'] == patient_id]
        
        # Limit cycles per patient
        selected_cycles = patient_cycles.head(max_cycles_per_patient)
        
        for _, cycle_row in selected_cycles.iterrows():
            try:
                print(f"\nProcessing {patient_id} - {cycle_row['series_id']}")
                
                # Load cycle
                cycle_4d, phase_files = load_breathing_cycle(cycle_row['cycle_dir'])
                
                # Analyze middle slice
                mid_slice = cycle_4d.shape[1] // 2
                motion_analysis = analyze_breathing_motion(cycle_4d, mid_slice)
                
                # Create video for middle slice
                cycle_name = Path(cycle_row['cycle_dir']).name
                video_path = EXPORTS_DIR / f"{patient_id}_{cycle_name}_batch.mp4"
                
                create_breathing_video_html(
                    cycle_4d,
                    slice_idx=mid_slice,
                    output_path=video_path,
                    fps=2
                )
                
                batch_results.append({
                    'patient_id': patient_id,
                    'cycle_id': cycle_row['cycle_id'],
                    'motion_magnitude': motion_analysis['avg_motion_magnitude'],
                    'video_path': str(video_path),
                    'status': 'success'
                })
                
            except Exception as e:
                print(f"Error processing {patient_id}: {e}")
                batch_results.append({
                    'patient_id': patient_id,
                    'cycle_id': cycle_row['cycle_id'],
                    'status': 'error',
                    'error': str(e)
                })
    
    # Save batch results
    batch_df = pd.DataFrame(batch_results)
    batch_results_path = EXPORTS_DIR / f"batch_results_{datetime.now():%Y%m%d_%H%M%S}.csv"
    batch_df.to_csv(batch_results_path, index=False)
    
    print(f"\nBatch processing complete: {batch_results_path}")
    return batch_df
